In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [1]:
param_date = '20240706'

In [2]:
import os

list_file = sorted(list(os.walk(f'./{param_date}/4.model_2/'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['make_1.AC.xlsx',
 'make_10.American LaFrance.xlsx',
 'make_100.Lamborghini.xlsx',
 'make_101.Lancia.xlsx',
 'make_102.Land Rover.xlsx',
 'make_103.Lexus.xlsx',
 'make_104.Lincoln.xlsx',
 'make_105.Lion.xlsx',
 'make_106.Lotus.xlsx',
 'make_107.MG.xlsx',
 'make_108.Mack.xlsx',
 'make_109.Maple Leaf Truck.xlsx',
 'make_11.American Motors.xlsx',
 'make_110.Maserati.xlsx',
 'make_111.Maybach.xlsx',
 'make_112.Mazda.xlsx',
 'make_113.Mercedes-Benz.xlsx',
 'make_114.Mercury.xlsx',
 'make_115.Merkur.xlsx',
 'make_116.Metropolitan.xlsx',
 'make_117.Mini.xlsx',
 'make_118.Mitsubishi.xlsx',
 'make_119.Mitsubishi Fuso.xlsx',
 'make_12.Aston Martin.xlsx',
 'make_120.Mobility Ventures.xlsx',
 'make_121.Monaco Coach.xlsx',
 'make_122.Motor Coach Industries.xlsx',
 'make_123.Nash.xlsx',
 'make_124.National RV.xlsx',
 'make_125.Neoplan.xlsx',
 'make_126.New Flyer.xlsx',
 'make_127.Newmar.xlsx',
 'make_128.Nexus.xlsx',
 'make_129.Nissan.xlsx',
 'make_13.Asuna.xlsx',
 'make_130.Nova Bus Corporation.xl

In [3]:
for file in list_file:

    # = = = = = = = = = = = = = = =

    import pandas as pd

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./{param_date}/4.model_2/{file}',
                           header=0,
                           dtype=str).fillna('')

    input_['Year'] = input_['Year'].astype(int)
    input_['Make No'] = input_['Make No'].astype(int)
    input_['Model No'] = input_['Model No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =

    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            for b in range(666):
                try:
                    import requests
                    import json

                    import sys
                    sys.path.append('../00.Tools')
                    from crawler_configuration import get_header, get_proxy

                    data = {'cssClass': 'form-control',
                            'dropdownType': '3',
                            'selectedValue': '',
                            'vehicleAttributes': {'Make': f'''{input_.loc[a, 'Make']}''',
                                                  'MakeID': f'''{input_.loc[a, 'Make Code']}''',
                                                  'ModelID': f'''{input_.loc[a, 'Model Code']}''',
                                                  'SkinID': '1',
                                                  'Year': f'''{input_.loc[a, 'Year']}'''}}

                    resp = requests.post('https://www.dormanproducts.com/ApplicationGuideService.asmx/BuildDropdown',
                                         data=json.dumps(data),
                                         headers=get_header(**{'Content-Type': 'application/json; charset=UTF-8'}),
                                         proxies=get_proxy(),
                                         timeout=(10, 10)).text

                    # = = = = = = = = = = = = = = =

                    json_ = json.loads(resp.strip())

                    # = = = = = = = = = = = = = = =

                    from bs4 import BeautifulSoup
                    from lxml import etree

                    soup = BeautifulSoup(json_['d'].strip(), 'lxml')
                    html = etree.HTML(str(soup))

                    # = = = = = = = = = = = = = = =

                    list_type_code = html.xpath('//select[@id="parttypeDropdown"]/option/@value')[1:]

                    # = = = = = = = = = = = = = = =

                    if len(list_type_code) == 0:
                        continue

                    # = = = = = = = = = = = = = = =

                    list_type = html.xpath('//select[@id="parttypeDropdown"]/option/text()')[1:]

                    # = = = = = = = = = = = = = = =

                    df_temp = pd.DataFrame({'Year': input_.loc[a, 'Year'],
                                            'Make No': input_.loc[a, 'Make No'],
                                            'Make': input_.loc[a, 'Make'],
                                            'Make Code': input_.loc[a, 'Make Code'],
                                            'Model No': input_.loc[a, 'Model No'],
                                            'Model': input_.loc[a, 'Model'],
                                            'Model Code': input_.loc[a, 'Model Code'],
                                            'Type No': [i+1 for i in range(len(list_type))],
                                            'Type': list_type,
                                            'Type Code': list_type_code,
                                            'Type Url': [f'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q={type_}' for type_ in list_type]})

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                    # = = = = = = = = = = = = = = =

                    crawler_status = 'ok'

                    # = = = = = = = = = = = = = = =

                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            if crawler_status == 'error':
                df_temp = pd.DataFrame([{'Year': input_.loc[a, 'Year'],
                                         'Make No': input_.loc[a, 'Make No'],
                                         'Make': input_.loc[a, 'Make'],
                                         'Make Code': input_.loc[a, 'Make Code'],
                                         'Model No': input_.loc[a, 'Model No'],
                                         'Model': input_.loc[a, 'Model'],
                                         'Model Code': input_.loc[a, 'Model Code']}])

                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Year']} > {input_.loc[a, 'Model No']}.{input_.loc[a, 'Model']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(20):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Year', 'Make No', 'Model No', 'Type No'],
                                                    ascending=[False, True, True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./{param_date}/5.type_1/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['Year', 'Make No', 'Model No'],
                                                ascending=[False, True, True],
                                                ignore_index=True)
        output_error.to_excel(f'''./{param_date}/0.error/{file.removesuffix('.xlsx')}-type_1_error.xlsx''', index=False)
        print('爬虫存在error')
        print()
    print('Done ~')

[状态：ok，尝试次数： 34] - 98.97% - make_97.Kenworth > 1983 > 3.850
[剩余数量：0] - [当前时间：16:18:10]

输出ing...

Done ~
总数量：284

[状态：ok，尝试次数：  1] - 99.49% - make_98.Kia > 2024 > 15.Telluride
[剩余数量：264] - [当前时间：16:18:12]

[状态：ok，尝试次数：  1] - 99.49% - make_98.Kia > 2024 > 5.Forte5
[剩余数量：263] - [当前时间：16:18:12]

[状态：ok，尝试次数：  1] - 99.49% - make_98.Kia > 2025 > 1.Sportage
[剩余数量：262] - [当前时间：16:18:12]

[状态：ok，尝试次数：  1] - 99.49% - make_98.Kia > 2023 > 2.EV6
[剩余数量：261] - [当前时间：16:18:12]

[状态：ok，尝试次数：  1] - 99.49% - make_98.Kia > 2024 > 6.K3 (Mexico)
[剩余数量：260] - [当前时间：16:18:12]

[状态：ok，尝试次数：  1] - 99.49% - make_98.Kia > 2024 > 4.Forte
[剩余数量：259] - [当前时间：16:18:13]

[状态：ok，尝试次数：  1] - 99.49% - make_98.Kia > 2023 > 4.Forte5
[剩余数量：258] - [当前时间：16:18:13]

[状态：ok，尝试次数：  1] - 99.49% - make_98.Kia > 2024 > 11.Sonet (Mexico)
[剩余数量：257] - [当前时间：16:18:13]

[状态：ok，尝试次数：  1] - 99.49% - make_98.Kia > 2024 > 8.Niro
[剩余数量：256] - [当前时间：16:18:13]

[状态：ok，尝试次数：  1] - 99.49% - make_98.Kia > 2024 > 14.Sportage
[剩余数量：255] - [当前时间：